## Описание проекта

### Цели проекта

Появилась тенденция к тому, чтобы создавать приложения для тех, кто любит читать. Компания, где я работаю, решила быть на волне и купила крупный сервис для чтения книг по подписке.

Моя первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

1. Таблица books
   Содержит данные о книгах:

       book_id — идентификатор книги;
       author_id — идентификатор автора;
       title — название книги;
       num_pages — количество страниц;
       publication_date — дата публикации книги;
       publisher_id — идентификатор издателя.

2. Таблица authors
   Содержит данные об авторах:

       author_id — идентификатор автора;
       author — имя автора.

3. Таблица publishers
   Содержит данные об издательствах:

       publisher_id — идентификатор издательства;
       publisher — название издательства;

4. Таблица ratings
   Содержит данные о пользовательских оценках книг:

       rating_id — идентификатор оценки;
       book_id — идентификатор книги;
       username — имя пользователя, оставившего оценку;
       rating — оценка книги.

5. Таблица reviews
   Содержит данные о пользовательских обзорах:

       review_id — идентификатор обзора;
       book_id — идентификатор книги;
       username — имя автора обзора;
       text — текст обзора.

### Схема данных

![](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

### Задания
#### Посчитать, сколько книг вышло после 1 января 2000 года;
#### Для каждой книги посчитайте количество обзоров и среднюю оценку;
#### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Последовательность действий

#### Исследование таблиц — вывод первых строк;
#### SQL-запросы для решения каждого задания (1) и вывод результатов каждого запроса.
#### Выводы по каждой из задач

## Выполнение задач проекта

### Загрузка библиотек, задание параметров подключения и создание строки  соединения к БД, подключение к БД

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': '', # имя пользователя
            'pwd': '', # пароль
            'host': '',
            'port': , # порт подключения
            'db': ''} # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

In [4]:
engine = create_engine(connection_string)

In [5]:
#pd.io.sql.read_sql(query, con = engine)

### Исследование таблиц-вывод первых строк

In [6]:
query_books_fstring = ''' SELECT * FROM books LIMIT 1 ''' 
query_authors_fstring = ''' SELECT * FROM authors LIMIT 1 '''
query_publishers_fstring = ''' SELECT * FROM publishers LIMIT 1 '''
query_ratings_fstring = ''' SELECT * FROM ratings LIMIT 1 '''
query_reviews_fstring = ''' SELECT * FROM reviews LIMIT 1 '''

In [7]:
books_fstring = pd.io.sql.read_sql(query_books_fstring, con = engine)
books_fstring

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [8]:
books_authors_fstring = pd.io.sql.read_sql(query_authors_fstring, con = engine)
books_authors_fstring

,author_id,author
0,1,A.S. Byatt


In [9]:
books_publishers_fstring = pd.io.sql.read_sql(query_publishers_fstring, con = engine)
books_publishers_fstring

,publisher_id,publisher
0,1,Ace


In [10]:
books_ratings_fstring = pd.io.sql.read_sql(query_ratings_fstring, con = engine)
books_ratings_fstring

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [11]:
books_reviews_fstring = pd.io.sql.read_sql(query_reviews_fstring, con = engine)
books_reviews_fstring

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


### Задача 1

Посчитать, сколько книг вышло после 1 января 2000 года

In [12]:
query_task_one = ''' SELECT COUNT(book_id)
                     FROM books
                     WHERE publication_date>'1999-12-31'
        ''' 
task_one=pd.io.sql.read_sql(query_task_one, con = engine)
print ('С начала двухтысячного года вышло(а) ', task_one.iloc[0,0], ' книг(а)')

С начала двухтысячного года вышло(а)  821  книг(а)


Если мы говорим о том, чтобы создать новый ресурс для чтения, то 821 книга (свежие) это по идее не много для выбора. Но не надо забывать об авторских отчислениях. И это наши вложения на старте магазина.

### Задача 2

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [13]:
query_test_books = ''' SELECT *
                            FROM books
                    '''
test_books = pd.io.sql.read_sql(query_test_books, con = engine)
test_books.info()
print ('Дубликатов в таблице test_books:', test_books.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
Дубликатов в таблице test_books: 0


In [14]:
query_task_two = ''' SELECT bo.title,
                            COUNT (re.review_id) AS reviews_count,
                            ROUND (AVG (ra.rating),2) AS avg_rating
                     FROM books AS bo
                     LEFT OUTER JOIN reviews AS re ON bo.book_id = re.book_id
                     LEFT OUTER JOIN ratings AS ra ON bo.book_id = ra.book_id
                     GROUP BY bo.book_id
        ''' 
task_two=pd.io.sql.read_sql(query_task_two, con = engine)
task_two.info()
print ('КОЛИЧЕСТВО ОБЗОРОВ И СРЕДНЯЯ ОЦЕНКА ДЛЯ КАЖДОЙ КНИГИ')
display(task_two.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          1000 non-null   object 
 1   reviews_count  1000 non-null   int64  
 2   avg_rating     1000 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 23.6+ KB
КОЛИЧЕСТВО ОБЗОРОВ И СРЕДНЯЯ ОЦЕНКА ДЛЯ КАЖДОЙ КНИГИ


,title,reviews_count,avg_rating
0,The Body in the Library (Miss Marple #3),4,4.50
1,Galápagos,4,4.50
2,A Tree Grows in Brooklyn,60,4.25
3,Undaunted Courage: The Pioneering First Missio...,4,4.00
4,The Prophet,28,4.29
5,American Gods (American Gods #1),70,4.00
6,Influence: The Psychology of Persuasion,9,4.33
7,The Idiot,12,4.25
8,Rebecca,28,4.00
9,The Source,4,3.50


Мы можем ранжировать книги по среднему рейтингу (оценка) и количеству отзывов (интерес и резонанс). И пополнять наш ресурс, используя сначала наиболее рейтинговые и обсуждаемые. В итоговый запрос вошло на 1 книгу меньше, чем в исходный файл. 

### Задача 3
Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [15]:
query_task_three = ''' SELECT pu.publisher
                             
                       FROM publishers AS pu
                       LEFT OUTER JOIN books AS bo ON pu.publisher_id = bo.publisher_id
                       WHERE bo.num_pages>50
                       GROUP BY pu.publisher_id
                       ORDER BY SUM(bo.book_id) DESC
                       LIMIT 1
        ''' 
task_three=pd.io.sql.read_sql(query_task_three, con = engine)
display(task_three)
print ('Издательство, которое выпустило наибольшее число книг толще 50 страниц: ', task_three.iloc[0,0])

,publisher
0,Penguin Books


Издательство, которое выпустило наибольшее число книг толще 50 страниц:  Penguin Books


Использование этого отбора позволяет нам ориентироваться на издательства (в первую очередь), которые выпускают наибольшее количество литературы (не брошюр). Penguin Books - одно из ведущих издательств, выпускающих литературу на английском языке.

### Задача 4
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [16]:
query_task_four = ''' SELECT au.author,
                             ROUND(AVG (ra.rating),2)
                      FROM authors AS au
                      JOIN books AS bo ON bo.author_id=au.author_id
                      JOIN ratings AS ra ON bo.book_id=ra.book_id
                      WHERE bo.book_id IN (SELECT book_id
                                           FROM ratings
                                           GROUP BY book_id
                                           HAVING COUNT(rating_id)>50)
                      GROUP BY au.author_id
                      ORDER BY AVG (ra.rating) DESC 
                      LIMIT 1
        ''' 
task_four=pd.io.sql.read_sql(query_task_four, con = engine)
display(task_four)
print ('Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками): ', task_four.iloc[0,0])

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками):  J.K. Rowling/Mary GrandPré


Использование этого отбора позволяет нам начать с приобретения производений наиболее рейтинговых авторов. Джоан Роулинг вполне логична.

### Задача 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [17]:
query_task_five = ''' WITH 
                      i AS (SELECT re.username, 
                            COUNT(re.review_id)
                            FROM reviews AS re
                            WHERE re.username IN (SELECT username
                                           FROM ratings
                                           GROUP BY username
                                           HAVING COUNT(rating_id)>50)
                            GROUP BY re.username)
                      SELECT ROUND(SUM (count)/COUNT(username),2) AS avg_reviews
                      FROM i
        ''' 
task_five=pd.io.sql.read_sql(query_task_five, con = engine)
display(task_five)
print ('Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок: ', task_five.iloc[0,0])

,avg_reviews
0,24.33


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок:  24.33


Этот отбор позволяет нам сформировать понимание аудитории наиболее активных пользователей, чтобы начать продвижение нашего ресурса с их помощью (например, вставлять в отзывы упоминание.

## Выводы
1. Отбор выпущенных только за последнее время книг позволяет минимизировать начальные расходы стартапа на закупку авторских прав.
2. Отборы позволяют нам выбрать наиболее позитивно воспринятые и обсуждаемые книги, чтобы начать с приобретения именно их (минимизация стартового бюджета).
3. Также мы можем выбрать наиболее крупные издательства, чтобы начать переговоры именно с ними (оптимизация усилий).
4. У них можно выкупать права в первую очередь на выходящие новинки самых рейтинговых авторов (чтобы поддерживать интерес аудитории.
5. Мы можем отобрать наиболее активных пользователей, чтобы начать продвижение именно с их помощью.
6. Как основную идею (слоган нового стартапа) можно сформулировать: Специально для вас мы следим за новинками и отбираем только лучшее, чтобы вы не тратили свое время.